# PD 6 Szymon Rećko

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from skimage.transform import rotate,swirl

## Dane

In [ ]:
data=fetch_olivetti_faces()

In [ ]:
data.keys()

## Przydatne funkcje

In [ ]:
def cal_RMSE(original_data, data_after_trans):
    for i in range(4):
        id = ids[i]*10
        print((mean_squared_error(original_data[id], data_after_trans[id], squared=False)))

In [ ]:
def RMSE_mod(modificated_image):
    transformed_data  = pca.transform(modificated_image)
    inverted = pca.inverse_transform(transformed_data)
    cal_RMSE(data_copy,inverted)

## 0. Rysowanie

In [ ]:
ids=[1,2,3,4]
images=data.images
fig_original, ax=plt.subplots(nrows=1, ncols=4, figsize=(18, 9))
ax=ax.flatten()
for i in range(4):
    id =ids[i]
    image_index=id*10
    ax[i].imshow(images[image_index], cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title("id:{}".format(id))

## 1. PCA

In [ ]:
data_copy=data.data.copy()
target=data.target.copy()
images=data.images.copy()
pca=PCA()
pca.fit(data_copy)
plt.figure(1, figsize=(12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
#wybieramy tyle komponentów by co najmniej 95% wariancji było wyjaśnionej
pca = PCA(n_components = 0.95)
data_pca = pca.fit_transform(data_copy)
kompresja=4096/data_pca.shape[1]
print("Stopień kompresji: "+str(kompresja))

## 2. Transformacja odwrotna

In [ ]:
inverted = pca.inverse_transform(data_pca)
inverted.shape
inverted_img = inverted.reshape(400,64,64)
inverted_img.shape

In [ ]:
images=inverted_img
fig_pca, ax=plt.subplots(nrows=1, ncols=4, figsize=(18, 9))
ax=ax.flatten()
for i in range(4):
    id =ids[i]
    image_index=id*10
    ax[i].imshow(images[image_index], cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title("id:{}".format(id))

In [ ]:
fig_original

In [ ]:
cal_RMSE(data_copy,inverted)

## 3. Przekształcone obrazy

In [ ]:
plt.figure(1, figsize=(20, 20))
for i in range(4):
    id = ids[i]*10
    #obrócone
    rotated = rotate(images[id], 90)
    plt.subplot(11, 5, i+1)
    plt.imshow(rotated, cmap=plt.cm.gray, interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
    #odbicie lustrzane
    horizontal_flip = images[id][:, ::-1]
    plt.subplot(11, 5, 5+i+1)
    plt.imshow(horizontal_flip, cmap=plt.cm.gray, interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
    #swirl
    swirled = swirl(images[id], rotation=0, strength=2, radius=90)
    plt.subplot(11, 5, 10+i+1)
    plt.imshow(swirled, cmap=plt.cm.gray, interpolation='nearest')
    plt.xticks(())
    plt.yticks(())

## 4. Transformacja, odwrotna transformacja, RMSE

In [ ]:
rotated = rotate(data_copy, 90)
print('RMSE dla obrócenia:')
RMSE_mod(rotated)
horizontal_flip = data_copy[:, ::-1]
print('RMSE dla odbicia:')
RMSE_mod(horizontal_flip)
swirled = swirl(data_copy, rotation=0, strength=2, radius=90)
print('RMSE dla swirled:')
RMSE_mod(swirled)

## 5. Wnioski

Na podstawie tego badania widać, że ta metoda może być użyta przy wykrywaniu anomali na zdjęciach takich jak odbicie czy obrócenie ponieważ jest tam zauważalnie większe RMSE, jednak nie jest w stanie zauważyć przekształcenia swirl.